In [1]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import sys


In [10]:
# Construct the argument parser and parse the arguments
sys.argv = ['mobilenetv3.ipynb', '-d', 'dataset']

ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
    help="path to input dataset")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
    help="path to output loss/accuracy plot")
ap.add_argument("-m", "--model", type=str,
    default="mask_detector.model",
    help="path to output face mask detector model")
args = vars(ap.parse_args())

In [11]:
# Initialize the initial learning rate, number of epochs to train for, and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [12]:
# Print a message indicating that images are being loaded
print("[INFO] loading images...")

# Grab the list of image paths
imagePaths = list(paths.list_images(args["dataset"]))

# Initialize the lists to store image data and corresponding labels
data = []
labels = []


# Loop over each image path
for imagePath in imagePaths:
    
    # Extract the class label from the directory name (second last part of the path)
    label = imagePath.split(os.path.sep)[-2]
    
    # Load the input image, resize it to 224x224 pixels, and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    
    # Append the processed image to the data list
    data.append(image)
    
    # Append the corresponding label to the labels list
    labels.append(label)
    
# Convert the lists of data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

[INFO] loading images...


D:\Applications\Python\Anaconda\Anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
# Perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Partition the data into training and testing splits
# Use 80% of the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

# Construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode="nearest")

In [6]:
# Load the MobileNetV3Large network, ensuring the fully connected (FC) layer sets are left off
baseModel = MobileNetV3Large(weights=None, include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

# Load the weights from the local file
weights_path = 'models/weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5'
baseModel.load_weights(weights_path)

# Construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


# Place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)


# Loop over all layers in the base model and freeze them so they will not be updated
for layer in baseModel.layers:
    layer.trainable = False

In [7]:
# Print a message indicating that the model is being compiled
print("[INFO] compiling model...")

# Initialize the Adam optimizer with the specified learning rate
opt = Adam(learning_rate=INIT_LR)

# Compile the model with binary cross-entropy loss and the Adam optimizer
# Also specify that we want to track accuracy during training
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Print a message indicating that the head of the network is being trained
print("[INFO] training head...")

# Train the head of the network
# Use the data augmentation generator to provide augmented training data
# Specify the number of steps per epoch and validation steps
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Epoch 1/20


220/220 ━━━━━━━━━━━━━━━━━━━━ 306s 1s/step - accuracy: 0.6676 - loss: 0.6431 - val_accuracy: 0.8242 - val_loss: 0.3696
Epoch 2/20


220/220 ━━━━━━━━━━━━━━━━━━━━ 31s 138ms/step - accuracy: 0.8567 - loss: 0.3507 - val_accuracy: 0.9375 - val_loss: 0.1964
Epoch 3/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.8820 - loss: 0.2792 - val_accuracy: 0.9531 - val_loss: 0.1383
Epoch 4/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 30s 135ms/step - accuracy: 0.9148 - loss: 0.2196 - val_accuracy: 0.9492 - val_loss: 0.1306
Epoch 5/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.9213 - loss: 0.2006 - val_accuracy: 0.9688 - val_loss: 0.0863
Epoch 6/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 31s 139ms/step - accuracy: 0.9291 - loss: 0.1885 - val_accuracy: 0.9694 - val_loss: 0.0812
Epoch 7/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 233s 1s/step - accuracy: 0.9307 - loss: 0.1793 - val_accuracy: 0.9746 - val_loss: 0.0728
Epoch 8/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 31s 138ms/step - accuracy: 0.9498 - loss: 0.1517 - val_accuracy: 0.9792 - val_loss: 0.0645
Epoch 9/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.9556 - loss: 0.1334 - val_accurac

In [8]:
# Print a message indicating that the network is being evaluated
print("[INFO] evaluating network...")

# Make predictions on the testing set
predIdxs = model.predict(testX, batch_size=BS)

# For each image in the testing set, find the index of the label with the highest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# Print a classification report showing the performance of the model
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

# Print a message indicating that the mask detector model is being saved
print("[INFO] saving mask detector model...")

# Save the trained model to disk
model.save("models/mask_detector_model.keras")

[INFO] evaluating network...
56/56 ━━━━━━━━━━━━━━━━━━━━ 34s 556ms/step
              precision    recall  f1-score   support

   with_mask       0.99      1.00      0.99      138
without_mask       1.00      0.99      0.99      138

    accuracy                           0.99      276
   macro avg       0.99      0.99      0.99      276
weighted avg       0.99      0.99      0.99      276

[INFO] saving mask detector model...
